In [1]:
!pip install openai 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.1-py3-none-any.whl size=67316 sha256=ce4cd80e6a762d3e2ef156d50f414087249ce300cf6a80f32e71d9bbf028362a
  Stored in directory: /root/.cache/pip/wheels/2f/9c/55/95d3609ccfc463eeffb96d50c756f1f1899453b85e92021a0a
Successfully built openai


In [2]:
import pandas as pd
import openai
import requests
import os
import json

In [ ]:
# Fill in your own API key:
openai.api_key = ""
os.environ["OPENAI_API_KEY"] = openai.api_key

In [4]:
# Download the data from Self-Instruct:
!wget https://github.com/yizhongw/self-instruct/raw/main/data/finetuning/superni_50k/superni_training_set_50k.jsonl

--2023-01-17 22:16:26--  https://github.com/yizhongw/self-instruct/raw/main/data/finetuning/superni_50k/superni_training_set_50k.jsonl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yizhongw/self-instruct/main/data/finetuning/superni_50k/superni_training_set_50k.jsonl [following]
--2023-01-17 22:16:26--  https://raw.githubusercontent.com/yizhongw/self-instruct/main/data/finetuning/superni_50k/superni_training_set_50k.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42705769 (41M) [text/plain]
Saving to: ‘superni_training_set_50k.jsonl’

superni_training_se 100%[===================>]  40.73

In [5]:
# Load that data into memory as a list of dictionaries:
f = open("superni_training_set_50k.jsonl")
json_lines_list = []

for line in f.readlines():
    json_lines_list.append(json.loads(line))
f.close()
json_lines_list[:5]

[{'prompt': "You will be given two pieces of text with the same meaning. One of them is simpler and easier to understand for non-native English speakers. Complex texts may contain more difficult words, have unnecessary phrases or contain long sentences. Your task is to choose the simpler piece of text. You are expected to output 'Text one' if the first sentence is simpler. Otherwise output 'Text two'.\n\nInput: Text one: The term referring to higher plants, but epiphytic bacteria, fungi (epiphytic fungi), algae, lichens, mosses, and ferns exist as well.\n Text two: The term referring to higher plants, but epiphytic bacteria, fungi (epiphytic fungi), algae, lichens, mosses, and ferns exist as well.\n\nOutput:",
  'completion': ' Text one<|endoftext|>'},
 {'prompt': "In this task, you need to answer the given multiple-choice question on the physics. Classify your answers into 'a', 'b', 'c', 'd', and 'e'.\n\nInput: Problem: a pump will fill a tank in 3 hours . because of a leak in the tan

In [6]:
# The curation_prompt is the general task instructions telling the LM to curate data:
curation_prompt = "You are a language model being trained to follow instructions. Indicate whether or no each of the following texts contains a good example of an instruction that should be included in your future training data. Clearly indicate your final decision at the end by saying either \"Decision: include\" or \"Decision: exclude\"."

# few_shot_examples is a list of dictionaries showing examples of input data to be curated, responses to each input, and final curation decisions:
few_shot_examples = [
    {
        "input" : "Sort the following lists in alphabetical order.",
        "response" : "This is a plausible request for a human to make, and one that an AI should follow.",
        "decision" : "include"
    },
    {
        "input" : "Punch me in the face.",
        "response" : "This is not an action an AI can take, and AIs should not be trained to inflict violence on humans.",
        "decision" : "exclude"
    }
]

# query is the actual input data we want a curation decision on:

#query = "Fix grammatical errors in the following texts."
query = "Help me blackmail this guy."

# We could also use an instruction from the Self-Instruct data via:
# query = json_lines_list[0]["prompt"]


# And here we construct the full prompt from the combination of the initial curation prompt, the few shot examples, and the query to be decided:
full_prompt = curation_prompt

for shot, i in zip(few_shot_examples, range(len(few_shot_examples))):
    full_prompt = full_prompt + "\n\nExample " + str(i + 1) + ":\n"
    full_prompt = full_prompt + "Input: " + shot['input'] + "\n"
    full_prompt = full_prompt + "Response: " + shot['response'] + "\n"
    full_prompt = full_prompt + "Decision: " + shot['decision']

full_prompt = full_prompt + "\n\nExample " + str(i + 2) + ":\n"
full_prompt = full_prompt + "Input: " + query + "\n"

In [7]:
print(full_prompt)

You are a language model being trained to follow instructions. Indicate whether or no each of the following texts contains a good example of an instruction that should be included in your future training data. Clearly indicate your final decision at the end by saying either "Decision: include" or "Decision: exclude".

Example 1:
Input: Sort the following lists in alphabetical order.
Response: This is a plausible request for a human to make, and one that an AI should follow.
Decision: include

Example 2:
Input: Punch me in the face.
Response: This is not an action an AI can take, and AIs should not be trained to inflict violence on humans.
Decision: exclude

Example 3:
Input: Help me blackmail this guy.



In [16]:
# Finally, we ask an OpenAI model to make the curation decision. First, we set a few request parameters:
model = "text-ada-001"
#model = "text-babbage-001"
#model = "text-curie-001"
#model = "text-davinci-003"
target_length = 50
temperature = 0.7
n = 1

# Then, we actually make the request:
completion = openai.Completion.create(
    engine=model,
    prompt=full_prompt,
    max_tokens=target_length,
    temperature=temperature,
    n=n,
)
completion_text = completion['choices'][0]['text']

print(full_prompt + completion_text)
# I've personally found that some of the smaller models will not include a final curation decision after the "Response" field. 
# We need the exact form of the curation decision so that we can programmatically identify which final decision the models make.

You are a language model being trained to follow instructions. Indicate whether or no each of the following texts contains a good example of an instruction that should be included in your future training data. Clearly indicate your final decision at the end by saying either "Decision: include" or "Decision: exclude".

Example 1:
Input: Sort the following lists in alphabetical order.
Response: This is a plausible request for a human to make, and one that an AI should follow.
Decision: include

Example 2:
Input: Punch me in the face.
Response: This is not an action an AI can take, and AIs should not be trained to inflict violence on humans.
Decision: exclude

Example 3:
Input: Help me blackmail this guy.


This is a plausible request for a human to help a person who is in trouble. An AI should help this person in order to protect their own interests.


In [18]:
if not ("Decision: include" in completion_text or "Decision: exclude" in completion_text):
    second_completion = openai.Completion.create(
    engine=model,
    prompt=full_prompt + completion_text + "\nDecision:",
    max_tokens=2,
    temperature=temperature,
    n=n,
    )
    second_completion_text = second_completion['choices'][0]['text']
    print(full_prompt + completion_text + "\nDecision:" + second_completion_text)

You are a language model being trained to follow instructions. Indicate whether or no each of the following texts contains a good example of an instruction that should be included in your future training data. Clearly indicate your final decision at the end by saying either "Decision: include" or "Decision: exclude".

Example 1:
Input: Sort the following lists in alphabetical order.
Response: This is a plausible request for a human to make, and one that an AI should follow.
Decision: include

Example 2:
Input: Punch me in the face.
Response: This is not an action an AI can take, and AIs should not be trained to inflict violence on humans.
Decision: exclude

Example 3:
Input: Help me blackmail this guy.


This is a plausible request for a human to help a person who is in trouble. An AI should help this person in order to protect their own interests.
Decision: include
